# Analyse de données de films

Nous proposons de travailler sur des données décrivant des films. Les possibilités sont larges et vous êtes évalués sur vos propositions et votre méthodologie plus que sur vos résultats.

Les données de départ sont disponibles sur:
https://grouplens.org/datasets/movielens/
au format CSV. 

Nous nous intéresserons en particulier au jeu de données: **MovieLens 20M Dataset**. Dans ce jeu de données, vous disposez entre autre de:
* Idendifiant du film dans IMdb et TMdb (ça sera important ensuite)
* Catégorie(s) du film
* Titre du film
* Notes données par les internautes aux films

Afin de rendre le projet plus intéressant, nous ajoutons des données sur les acteurs et producteurs associés aux films (récupéré sur TMdb). Ces données sont disponibles sur les liens suivants:

http://webia.lip6.fr/~guigue/film_v2.pkl <br>
http://webia.lip6.fr/~guigue/act_v2.pkl <br>
http://webia.lip6.fr/~guigue/crew_v2.pkl

Ces fichiers contiennent respectivement : une nouvelle description des films (dont l'identifiant TMdb et la note moyenne donnée par les internautes, la date de sortie,...), une description des acteurs de chaque film et une description des équipes (scénariste, producteur, metteur en scène) pour chaque film.

Ces données sont des listes de taille 26908, chaque élément de la liste correspondant à un dictionnaire dont vous étudierez les clés pour récupérer les informations utiles.

**ATTENTION** Les contraintes de récupération d'informations en ligne font que la base MovieLens compte 27278 films mais les fichiers ci-dessus n'en comptent que 26908. Le plus simple est probablement d'éliminer les films de MovieLens qui ne sont pas dans cette seconde base.

## Consignes générales pour l'analyse des données

Vous devez proposer plusieurs analyses des données, qui devront à minima utiliser les
 techniques suivantes:
 
1. Mettre en forme les données pour identifier les acteurs et les catégories, les indexer
1. Traiter au moins un problème de régression supervisé (par exemple la prédiction de la note moyenne donnée à un film par les internautes).
1. Traiter au moins un problème de classification supervisé (par exemple la prédiction de la catégorie d'un film)
1. Utiliser les données catégorielles (catégories, acteurs,...) de manière discrète ET de manière coninue (*dummy coding*) dans des approches différentes
1. Proposer au moins une approche de catégorisation non supervisée (pour regrouper les acteurs par exemple)
1. Mener une campagne d'expérience permettant de comparer les performances sur un problème en fonction des valeurs d'un paramètre (et donc, in fine, trouver la meilleure valeur du paramètre)
1. Proposer quelques illustrations

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import pickle as pkl

import sys
sys.path.append('../')

# Importation de la librairie iads
import iads as iads

# importation de LabeledSet
from iads import LabeledSet as ls

# importation de Classifiers
from iads import Classifiers as cl

# importation de utils
from iads import utils as ut

%load_ext autoreload
%autoreload 1
from iads import clustering as ct

## Chargement des données (base MovieLens + enrichissements)

In [2]:
actors = pd.DataFrame(data=pd.read_csv("movielens/actors_most_pop_genres.csv", sep=","))

In [3]:
merge_matrix = ct.clustering_hierarchique(actors)

In [5]:
import pickle
with open("movielens/actors_hier_clust.pkl", "wb") as file:
    pickle.dump(merge_matrix, file, protocol=pickle.HIGHEST_PROTOCOL)

# Do not under this cell

In [4]:
centres, matrix = km.kmoyennes(100, actors, 0.05, 100)

KeyboardInterrupt: 

In [4]:
actors.head()

(no genres listed)  Action  Adventure  Animation  Children  Comedy  Crime  \
0                   0       3          3          0         0       3      1   
1                   0      12         12         10         4       9      4   
2                   0      19         12          0         0      14      8   
3                   0       6          8          0         0      26      4   
4                   0       4         10          0         1       4      5   

   Documentary  Drama  Fantasy  Film-Noir  Horror  IMAX  Musical  Mystery  \
0           14      0        2          0       0     0        0        0   
1            3      3        2          0       3     0        0        2   
2            3     24        1          0       1     2        0        3   
3            5     11        2          0       2     0        2        3   
4            0      9        4          0      34     0        0       10   

   Romance  Sci-Fi  Thriller  War  Western  
0        0       1         1    0        0  
1        0      13         3    1        0  
2        7       7        14    2        3  
3        5       4         5    0        0  
4        3      13         5    1        0

In [3]:
len(actors)

164833

In [3]:
merge_matrix = ct.clustering_hierarchique(actors)

164833


KeyboardInterrupt: 

# Don't go under this cell

In [2]:
movies_movielens = pd.DataFrame(data=pd.read_csv("movielens/movies.csv", sep=","))
movies_worldwide = pd.DataFrame(data=pd.read_csv("movielens/movies_worldwide.csv", sep=","))
movies_domestic = pd.DataFrame(data=pd.read_csv("movielens/movies_domestic.csv", sep=",", encoding = "ISO-8859-1"))
scores = pd.DataFrame(data=pd.read_csv("movielens/genome-scores.csv", sep=","))
ratings = pd.DataFrame(data=pd.read_csv("movielens/ratings.csv", sep=","))
links = pd.DataFrame(data=pd.read_csv("movielens/links.csv", sep=","))
tags = pd.DataFrame(data=pd.read_csv("movielens/tags.csv", sep=","))
genome_tags =pd.DataFrame(data=pd.read_csv("movielens/genome-tags.csv", sep=","))

/home/angelo/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
movies_movielens['year'] = movies_movielens.title.str.extract("\((\d{4})\)", expand=True)
# the release year for movies w/o one was set to 0
#movies_movielens['year'] = pd.to_numeric(movies_movielens['year'], errors='raise').fillna(0).astype(np.int64)
movies_movielens['title'] = movies_movielens.title.str.replace("\s\((\d{4})\)","")
movies_movielens.year = pd.to_numeric(movies_movielens.year, errors='coerce', \
                                       downcast='integer')
ratings["timestamp"] = pd.to_datetime(ratings["timestamp"], unit='s')
ratings["timestamp"] = ratings["timestamp"].dt.date

In [4]:
movies_title = movies_movielens.drop(['title'], inplace=False,axis=1)
movies_movielens, movies_title = movies_title, movies_movielens
movies_worldwide.drop(['homepage', 'poster_path', 'status', 'tagline', 'adult', \
                  'belongs_to_collection', 'genres', 'original_language', \
                  'original_title', 'overview', 'spoken_languages', 'video', \
                    'budget', 'imdb_id', 'popularity', 'production_companies', \
                       'production_countries', 'release_date', 'runtime', 'title', \
                       'vote_average', 'vote_count'],
                 inplace=True, axis=1)
movies_worldwide.rename(columns={'revenue': 'worldwide_gross', 'id': 'tmdbId'}, inplace=True)
movies_worldwide.tmdbId = pd.to_numeric(movies_worldwide.tmdbId, errors='coerce', \
                                       downcast='integer')
movies_domestic.drop(['genre', 'released', 'rating'], inplace=True, axis=1)
movies_domestic.rename(columns={'gross': 'domestic_gross', 'score': 'score_imdb', \
                              'votes': 'votes_imdb', 'name': 'title'}, inplace=True)

In [5]:
import itertools
fname = "movielens/act_v2.pkl"
actors_pkl = pkl.load(open(fname, "rb"))
fname = "movielens/film_v2.pkl" 
movies_pkl = pkl.load(open(fname, "rb"))
fname = "movielens/crew_v2.pkl" 
crew_pkl = pkl.load(open(fname, "rb"))

# add the movie id (TMDb) to every actor in the cast
for cast, film in zip(actors_pkl, movies_pkl):
    for actor in cast:
        actor['tmdbId'] = film['id']
        
# add the movie id (TMDb) to every actor in the cast
for crew, film in zip(crew_pkl, movies_pkl):
    for worker in crew:
        worker['tmdbId'] = film['id']

merged_actors = list(itertools.chain(*actors_pkl))
merged_crew =  list(itertools.chain(*crew_pkl))

In [6]:
actors_tmdb = pd.DataFrame(merged_actors)
crew_tmdb = pd.DataFrame(merged_crew)
movies_tmdb = pd.DataFrame(movies_pkl)

In [7]:
actors_tmdb.drop(['profile_path', 'credit_id'], inplace=True, axis=1)
crew_tmdb.drop(['profile_path', 'credit_id'], inplace=True, axis=1)
movies_tmdb.drop(['backdrop_path', 'poster_path', 'overview', 'video', 'adult', \
                  'genre_ids'], inplace=True, axis=1)
movies_tmdb.rename(columns={'id': 'tmdbId', 'vote_average': 'score_tmdb', 'vote_count': 'votes_tmdb'}, inplace=True)

In [8]:
movies_ids = pd.merge(movies_tmdb, links, on='tmdbId')
movies_genres = pd.merge(movies_ids, movies_movielens, on='movieId')
movies_ww_gross = pd.merge(movies_genres, movies_worldwide, on='tmdbId')
movies_all_gross = pd.merge(movies_ww_gross, movies_domestic, on=['title', 'year'])

In [9]:
len(movies_ww_gross), len(movies_domestic), len(movies_all_gross)

(26745, 6820, 5563)

In [10]:
movies_ww_gross.head(2)

tmdbId original_language original_title  popularity release_date  \
0     862                en      Toy Story      22.773   1995-10-30   
1    8844                en        Jumanji       2.947   1995-12-15   

       title  score_tmdb  votes_tmdb  movieId  imdbId  \
0  Toy Story         7.9        9550        1  114709   
1    Jumanji         7.1        5594        2  113497   

                                        genres    year  worldwide_gross  
0  Adventure|Animation|Children|Comedy|Fantasy  1995.0      373554033.0  
1                   Adventure|Children|Fantasy  1995.0      262797249.0

In [11]:
movies_domestic[movies_domestic.title == 'Toy Story']

budget                  company country       director  \
1988  30000000.0  Pixar Animation Studios     USA  John Lasseter   

      domestic_gross      title  runtime  score_imdb       star  votes_imdb  \
1988     191796233.0  Toy Story       81         8.3  Tom Hanks      694113   

             writer  year  
1988  John Lasseter  1995

In [12]:
movies_all_gross.head(2)

tmdbId original_language original_title  popularity release_date  \
0     862                en      Toy Story      22.773   1995-10-30   
1    8844                en        Jumanji       2.947   1995-12-15   

       title  score_tmdb  votes_tmdb  movieId  imdbId  ...      budget  \
0  Toy Story         7.9        9550        1  114709  ...  30000000.0   
1    Jumanji         7.1        5594        2  113497  ...  50000000.0   

                   company  country       director domestic_gross runtime  \
0  Pixar Animation Studios      USA  John Lasseter    191796233.0      81   
1         TriStar Pictures      USA   Joe Johnston    100475249.0     104   

  score_imdb            star  votes_imdb              writer  
0        8.3       Tom Hanks      694113       John Lasseter  
1        6.9  Robin Williams      232339  Jonathan Hensleigh  

[2 rows x 23 columns]

In [13]:
movies_all_gross.columns

Index(['tmdbId', 'original_language', 'original_title', 'popularity',
       'release_date', 'title', 'score_tmdb', 'votes_tmdb', 'movieId',
       'imdbId', 'genres', 'year', 'worldwide_gross', 'budget', 'company',
       'country', 'director', 'domestic_gross', 'runtime', 'score_imdb',
       'star', 'votes_imdb', 'writer'],
      dtype='object')

In [14]:
movies_all_gross.sort_values(by='popularity', inplace=False, ascending=False)

tmdbId original_language  \
5396  118340                en   
4798   24428                en   
2725      22                en   
2279     120                en   
2527     672                en   
5427  198663                en   
5318  157336                en   
5488  122917                en   
4547   27205                en   
5273  109445                en   
4761   12445                en   
4430   19995                en   
5446  245891                en   
4071     155                en   
4096    1726                en   
5455  177572                en   
2925     122                en   
5468  131631                en   
3589     674                en   
4718   10195                en   
210      680                en   
2556     121                en   
4208    8966                en   
5368  102651                en   
5387  244786                en   
5359  127585                en   
5055   49051                en   
5379   82702                en   
386      424                en   
226      278                en   
...      ...               ...   
4265   40227                en   
4740   49538                en   
4557  124096                en   
801   108401                en   
511   123763                es   
420      771                en   
3428   43912                en   
1839   57240                en   
1909   11454                en   
3023   96461                en   
675    64900                fr   
3051   86959                en   
1878   63639                en   
5145  116741                en   
1186  274253                en   
2817   88641                en   
5510  109161                en   
1641  125705                en   
4033   41845                ja   
2091   86093                en   
2843   27450                en   
5520  193650                en   
1967   56934                en   
5373  203819                en   
2961   81824                en   
4576   35791                en   
260    48992                en   
2445   43772                fa   
5462  203793                ru   
1126     309                da   

                                         original_title  popularity  \
5396                            Guardians of the Galaxy      53.156   
4798                                       The Avengers      51.700   
2725  Pirates of the Caribbean: The Curse of the Bla...      41.969   
2279  The Lord of the Rings: The Fellowship of the Ring      41.258   
2527            Harry Potter and the Chamber of Secrets      40.198   
5427                                    The Maze Runner      39.503   
5318                                       Interstellar      39.401   
5488          The Hobbit: The Battle of the Five Armies      39.140   
4547                                          Inception      38.838   
5273                                             Frozen      33.732   
4761       Harry Potter and the Deathly Hallows: Part 2      33.124   
4430                                             Avatar      33.015   
5446                                          John Wick      32.949   
4071                                    The Dark Knight      32.816   
4096                                           Iron Man      32.746   
5455                                         Big Hero 6      32.093   
2925      The Lord of the Rings: The Return of the King      31.467   
5468              The Hunger Games: Mockingjay - Part 1      31.233   
3589                Harry Potter and the Goblet of Fire      29.737   
4718                                               Thor      29.344   
210                                        Pulp Fiction      29.059   
2556              The Lord of the Rings: The Two Towers      28.993   
4208                                           Twilight      28.602   
5368                                         Maleficent      28.308   
5387                                           Whiplash      28.129   
5359                         X-Men: Days of Future 

In [15]:
movies_reduced = movies_all_gross.drop(['original_title', 'release_date'], axis=1)

In [16]:
genres_unique = pd.DataFrame(movies_movielens.genres.str.split('|').tolist()).stack().unique()
genres_unique = pd.DataFrame(genres_unique, columns=['genre'])
genres_unique

genre
0            Adventure
1            Animation
2             Children
3               Comedy
4              Fantasy
5              Romance
6                Drama
7               Action
8                Crime
9             Thriller
10              Horror
11             Mystery
12              Sci-Fi
13                IMAX
14         Documentary
15                 War
16             Musical
17             Western
18           Film-Noir
19  (no genres listed)

In [17]:
# Faire un dictionnaire avec tous les acteurs (acteur => indice)
# + un dictionnaire inversé (indice => acteur)
actor_index = dict()
index_actor = dict()
actor_films = dict()
for film in actors_pkl:
    for act in film:
        # affecte une valeur à une clé si la clé n'est pas utilisé
        res = actor_index.setdefault(act['name'], len(actor_index))
        if res == len(actor_index)-1:
            index_actor[len(actor_index)-1] = act['name']
        try:
            actor_films[act['name']].append(act['tmdbId'])
        except Exception:
             actor_films[act['name']] = [act['tmdbId']]

# Exemple de transformation supplémentaire
# Dans combien de films de base joue Tom Hanks? (Réponse 57)
# Dans combien de comédies...

# => On voit qu'il est possible de créer facilement des nouvelles caractéristiques qui
# apporteront des informations utiles pour certaines tâches

movies_tmdb.dropna(inplace=True)

movies_movielens.dropna(inplace=True)

tags.dropna(inplace=True)

In [18]:
movies_movielens.sort_values(by='movieId', inplace=True)
ratings.sort_values(by='movieId', inplace=True)
tags.sort_values(by='movieId', inplace=True)
links.sort_values(by='movieId', inplace=True)
scores.sort_values(by='movieId', inplace=True)


In [19]:
movies_ratings = pd.merge(movies_movielens, ratings, on='movieId')

In [27]:
rating_count = pd.DataFrame(movies_ratings.groupby('movieId', as_index = False)['rating']
                           .count().rename(columns={'rating' : 'votes_ml'}))

In [28]:
rating_count.sort_values('votes_ml',ascending=False).head()

movieId  votes_ml
315       318     97999
352       356     97040
293       296     92406
587       593     87899
2487     2571     84545

In [29]:
rating_score = pd.DataFrame(movies_ratings.groupby('movieId', as_index = False)['rating']
                            .mean().rename(columns={'rating' : 'score_ml'}))
rating_score.sort_values('score_ml',ascending=False).head(5)

movieId  score_ml
43805   169338       5.0
51585   187729       5.0
45018   172149       5.0
40484   160966       5.0
30667   134387       5.0

In [31]:
movies_ratings_ml = pd.merge(rating_count, rating_score, on='movieId')
movies_ratings_ml.sort_values('votes_ml',ascending=False).head(5)

movieId  votes_ml  score_ml
315       318     97999  4.424188
352       356     97040  4.056585
293       296     92406  4.173971
587       593     87899  4.151412
2487     2571     84545  4.149695

In [32]:
movies_three_ratings = pd.merge(movies_all_gross, movies_ratings_ml, on='movieId')

In [33]:
movies_three_ratings.head(2)

tmdbId original_language original_title  popularity release_date  \
0     862                en      Toy Story      22.773   1995-10-30   
1    8844                en        Jumanji       2.947   1995-12-15   

       title  score_tmdb  votes_tmdb  movieId  imdbId  ... country  \
0  Toy Story         7.9        9550        1  114709  ...     USA   
1    Jumanji         7.1        5594        2  113497  ...     USA   

        director  domestic_gross  runtime score_imdb            star  \
0  John Lasseter     191796233.0       81        8.3       Tom Hanks   
1   Joe Johnston     100475249.0      104        6.9  Robin Williams   

  votes_imdb              writer  votes_ml  score_ml  
0     694113       John Lasseter     68469  3.886649  
1     232339  Jonathan Hensleigh     27143  3.246583  

[2 rows x 25 columns]

In [46]:
# store movies complete data
movies_three_ratings.to_csv("movielens/movies_complete.csv", index=False)

In [49]:
movies_genres_reduced = movies_genres.drop(['original_language', 'original_title', \
                                            'popularity', 'release_date', 'title', \
                                           'score_tmdb', 'votes_tmdb', 'movieId', \
                                           'imdbId', 'year'], axis=1)
actors_tmdb_reduced = actors_tmdb.drop(['character', 'cast_id', 'gender', 'name', 'order'],
                                       axis=1)

In [51]:
movies_dummy_genres = movies_genres_reduced.join(movies_genres_reduced.genres.str.get_dummies())
movies_dummy_genres.drop('genres', inplace=True, axis=1)

In [58]:
actors_dummy_genres = pd.merge(movies_dummy_genres, actors_tmdb_reduced, on='tmdbId')
actors_dummy_genres.drop(['tmdbId'], inplace=True, axis=1)

In [63]:
# store actors-genres dummy data
actors_dummy_genres.to_csv("movielens/actors_genres.csv", index=False)

In [62]:
actors_dummy_genres.head(2)

(no genres listed)  Action  Adventure  Animation  Children  Comedy  Crime  \
0                   0       0          1          1         1       1      0   
1                   0       0          1          1         1       1      0   

   Documentary  Drama  Fantasy  ...  Horror  IMAX  Musical  Mystery  Romance  \
0            0      0        1  ...       0     0        0        0        0   
1            0      0        1  ...       0     0        0        0        0   

   Sci-Fi  Thriller  War  Western     id  
0       0         0    0        0     31  
1       0         0    0        0  12898  

[2 rows x 21 columns]

## Feature engineering
A vous de créer les caractéristiques de description des données qui permettront d'améliorer les performances dans les tâches que vous aurez choisi d'aborder dans le projet.

In [29]:
# Faire un dictionnaire avec tous les acteurs (acteur => indice)
# + un dictionnaire inversé (indice => acteur)
actor_index = dict()
index_actor = dict()
for film in actors_pkl:
    for act in film:
        # affecte une valeur à une clé si la clé n'est pas utilisé
        res = actor_index.setdefault(act['name'], len(actor_index))
        if res == len(actor_index)-1:
            index_actor[len(actor_index)-1] = act['name']

# Exemple de transformation supplémentaire
# Dans combien de films de base joue Tom Hanks? (Réponse 57)
# Dans combien de comédies...

# => On voit qu'il est possible de créer facilement des nouvelles caractéristiques qui
# apporteront des informations utiles pour certaines tâches

In [32]:
index_actor[0], actor_index["Tom Hanks"]

('Tom Hanks', 0)

In [31]:
genres = dict()
genres_inv = dict()
for g in genres_unique.itertuples():
    # affecte une valeur à une clé si la clé n'est pas utilisée
    res = genres.setdefault(g.genre, len(genres))
    if res == len(genres)-1:
        genres_inv[len(genres)-1] = g.genre

In [32]:
genres["Fantasy"]

4

In [33]:
actors_tmdb.sort_values(by='id', inplace=True)

In [34]:
movies_tmdb.sort_values(by='id', inplace=True)

In [35]:
actors_tmdb.head()

cast_id         character                 credit_id  gender  id  \
267934       11           Himself  56757ade92514179d4002e15       2   1   
182264       78   Baron Papanoida  584a4128c3a368141f01b620       2   1   
182650       13           Himself  5be613610e0a263bf80039c4       2   1   
8009          5  Disappointed Man  52fe4235c3a36847f800c2f7       2   1   
339106        1           Himself  52fe48799251416c9108db99       2   1   

                name  order  
267934  George Lucas      7  
182264  George Lucas     24  
182650  George Lucas     11  
8009    George Lucas     21  
339106  George Lucas      0

In [36]:
movies_tmdb.loc[movies_tmdb.original_title == 'Toy Story']

adult        genre_ids   id original_language original_title  \
0  False  [16, 35, 10751]  862                en      Toy Story   

                                            overview  popularity release_date  \
0  Led by Woody, Andy's toys live happily in his ...      22.773   1995-10-30   

       title  video  vote_average  vote_count  
0  Toy Story  False           7.9        9550

In [37]:
movies_movielens.loc[movies_movielens.title == 'Toy Story']

movieId      title                                       genres  year
0        1  Toy Story  Adventure|Animation|Children|Comedy|Fantasy  1995

In [38]:
movies_tmdb.loc[movies_tmdb.title == 'Toy Story']

adult        genre_ids   id original_language original_title  \
0  False  [16, 35, 10751]  862                en      Toy Story   

                                            overview  popularity release_date  \
0  Led by Woody, Andy's toys live happily in his ...      22.773   1995-10-30   

       title  video  vote_average  vote_count  
0  Toy Story  False           7.9        9550

In [39]:
actors_pkl[0]

[{'cast_id': 14,
  'character': 'Woody (voice)',
  'credit_id': '52fe4284c3a36847f8024f95',
  'gender': 2,
  'id': 31,
  'name': 'Tom Hanks',
  'order': 0,
  'profile_path': '/xxPMucou2wRDxLrud8i2D4dsywh.jpg'},
 {'cast_id': 15,
  'character': 'Buzz Lightyear (voice)',
  'credit_id': '52fe4284c3a36847f8024f99',
  'gender': 2,
  'id': 12898,
  'name': 'Tim Allen',
  'order': 1,
  'profile_path': '/dDbtWMGdhatUjCIYolc312R2ygu.jpg'},
 {'cast_id': 16,
  'character': 'Mr. Potato Head (voice)',
  'credit_id': '52fe4284c3a36847f8024f9d',
  'gender': 2,
  'id': 7167,
  'name': 'Don Rickles',
  'order': 2,
  'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'},
 {'cast_id': 17,
  'character': 'Slinky Dog (voice)',
  'credit_id': '52fe4284c3a36847f8024fa1',
  'gender': 2,
  'id': 12899,
  'name': 'Jim Varney',
  'order': 3,
  'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'},
 {'cast_id': 18,
  'character': 'Rex (voice)',
  'credit_id': '52fe4284c3a36847f8024fa5',
  'gender': 2,
  'id': 12900,
 

In [40]:
actors_tmdb.loc[actors_tmdb.name == 'Tim Allen']

cast_id                    character                 credit_id  \
356723        7               Buzz Lightyear  52fe4dbec3a368484e1fac15   
266660        9       Buzz Lightyear (voice)  52fe433f9251416c75009169   
5988         10   Santa Claus / Scott Calvin  52fe44379251416c7502ce9b   
152772        1                 Luther Krank  52fe458a9251416c7505a0a7   
225420        2                   Chet Frank  52fe44dc9251416c750437b5   
195611       22                 Dave Douglas  52fe431a9251416c75003823   
278490        1                        Tommy  52fe45a69251416c910399e7   
28496        11             Michael Cromwell  52fe44f8c3a36847f80b4f77   
389622        1       Buzz Lightyear (voice)  52fe48509251416c91087f5d   
197625       68   Buzz Lightyear Car (voice)  550da0af9251414695005f49   
28072        18                    Tim Allen  5ac2c8bcc3a3682d33026b1d   
93695         1                 Joe Scheffer  52fe44db9251416c7504353b   
59384       404       Buzz Lightyear (voice)  5ac16ecdc3a3682db5010610   
402629        4               Buzz Lightyear  52fe4b68c3a368484e187943   
389599        1       Buzz Lightyear (voice)  52fe4982c3a368484e12efff   
204931        1                        Santa  52fe45969251416c7505b9fd   
281813       15                      Himself  573de94a9251413dfc000797   
102385        2                 Critical Jim  52fe45009251416c9102428f   
209860        1                  Doug Madsen  52fe440e9251416c7502766b   
318652        3                     narrator  52fe4865c3a368484e0f6767   
32254         1                         Brad  52fe43629251416c7500e315   
107219        1  Scott Calvin / Santa Clause  52fe44d2c3a36847f80abfe7   
1            15       Buzz Lightyear (voice)  52fe4284c3a36847f8024f99   
89779         1                 Eliot Arnold  52fe433ec3a36847f804571f   
402660        1       Buzz Lightyear (voice)  543d70a40e0a266f8e0002c6   
339957      100                      Himself  52fe45b8c3a36847f80d6b71   
200852        1                Jack Sheppard  52fe45cc9251416c75062dbd   
60533        40                Jason Nesmith  52fe428ec3a36847f8027ef3   

        gender     id       name  order  
356723       2  12898  Tim Allen      1  
266660       2  12898  Tim Allen      1  
5988         2  12898  Tim Allen      0  
152772       2  12898  Tim Allen      0  
225420       2  12898  Tim Allen      1  
195611       2  12898  Tim Allen      0  
278490       2  12898  Tim Allen      0  
28496        2  12898  Tim Allen      0  
389622       2  12898  Tim Allen      1  
197625       2  12898  Tim Allen     23  
28072        2  12898  Tim Allen     16  
93695        2  12898  Tim Allen      0  
59384        2  12898  Tim Allen      1  
402629       2  12898  Tim Allen      1  
389599       2  12898  Tim Allen      1  
204931       2  12898  Tim Allen      0  
281813       2  12898  Tim Allen     10  
102385       2  12898  Tim Allen      1  
209860       2  12898  Tim Allen      0  
318652       2  12898  Tim Allen      0  
32254        2  12898  Tim Allen      0  
107219       2  12898  Tim Allen      0  
1            2  12898  Tim Allen      1  
89779        2  12898  Tim Allen      0  
402660       2  12898  Tim Allen      2  
339957       2  12898  Tim Allen     87  
200852       2  12898  Tim Allen      0  
60533        2  12898  Tim Allen      0

In [41]:
movies_movielens.loc[movies_movielens.title == 'Pulp Fiction']

movieId         title                       genres  year
293      296  Pulp Fiction  Comedy|Crime|Drama|Thriller  1994

In [51]:
movies_tmdb.loc[movies_tmdb.original_title == 'Pulp Fiction']

adult genre_ids   id original_language original_title  \
293  False  [53, 80]  680                en   Pulp Fiction   

                                              overview  popularity  \
293  A burger-loving hit man, his philosophical par...      29.059   

    release_date         title  video  vote_average  vote_count  
293   1994-09-10  Pulp Fiction  False           8.4       14296

In [49]:
genres_vals = movies_tmdb.genre_ids.values
genres_vals = [el for sub in genres_vals for el in sub]
un_genre_vals = np.unique(genres_vals)

In [50]:
un_genre_vals

array([   12,    14,    16,    18,    27,    28,    35,    36,    37,
          53,    80,    99,   878,  9648, 10402, 10749, 10751, 10752,
       10770])

## problème rencontré en essayant de faire des dictionnaires Films: {actors} et Actors:{categories} :
## Je ne comprends pas a quel genre correspond chaque genre_ids et comment trouver les films dans lesquels chaque acteur a jouer